# 0427 시점 최고 성적(Cog 코드) 기반 전처리 + train test 무지성 합친 전처리

In [17]:
import pandas as pd
import os
import random
import numpy as np
from tqdm import tqdm

## 1. 데이터 로딩

In [2]:
data_dir = '/opt/ml/input/DKT/data' # 경로는 상황에 맞춰서 수정해주세요!

dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
} 

csv_file_path = os.path.join(data_dir, 'total_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
df = pd.read_csv(csv_file_path, dtype=dtype, parse_dates=['Timestamp']) 

df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [3]:
def get_month(timestamp):
    timestamp = str(timestamp)
    year_month = timestamp[:7]
    # After EDA, we found data of year 2019 are only composed of December 31th
    # thus setting it to January
    if year_month[:4] == '2019':
        return '01'
    return year_month[5:]

In [4]:
def get_hour(timestamp):
    timestamp = str(timestamp)
    return timestamp[-8:-6]

In [5]:
df['month'] = df.Timestamp.apply(get_month)
df['hour'] = df.Timestamp.apply(get_hour)
# df['week'] = df['Timestamp'].dt.isocalendar().week

In [6]:
diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
diff = diff.fillna(pd.Timedelta(seconds=0))
diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

In [7]:
df['duration'] = diff.values

mean_duration = diff[(diff <= 135) & (diff >= 0)].mean()

# criterion: quantile (75%) or NA => mean duration
df['duration'] = df.duration.apply(lambda x: x if x <= 135 else mean_duration)
df['duration'] = df.duration.apply(lambda x: x if x > 0 else mean_duration)

In [8]:
# answerCode -1 빼고 하기
ans = df.loc[:, ['userID', 'answerCode']].groupby('userID').rolling(window=3, closed='left').mean()
df['past_OX'] = ans.values
df['past_OX'] = df.past_OX.fillna(method='bfill')

In [9]:
df['assessment_1'] = df.assessmentItemID.apply(lambda  x: int(x[2]))
df['assessment_2'] = df.assessmentItemID.apply(lambda  x: int(x[4:7]))
df['assessment_3'] = df.assessmentItemID.apply(lambda  x: int(x[7:]))

In [10]:
knowledge_clustered = df.loc[:, ['userID', 'KnowledgeTag']].groupby('userID').rolling(window=3, closed='right').std()
df['knowledge_clustered'] = knowledge_clustered.values
df['knowledge_clustered'] = df.knowledge_clustered.fillna(method='bfill')

In [11]:
df['knowledge_clustered'][df.knowledge_clustered > 100] = 100

/tmp/ipykernel_17817/2551097806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['knowledge_clustered'][df.knowledge_clustered > 100] = 100


In [16]:
def past_testid_maker(df_in,num = 1):
    #유저 testid group 전처리
    df_in.groupby(['userID'])['testId'].unique()
    userIDs = df_in['userID'].unique()
    gp_uniq_u_tID = df_in.groupby(['userID'])['testId'].unique()
    grouped = df_in.groupby(['userID','testId'])
    
    # 실제 연산
    num = 5 # testId = shift 수
    for i in range(1,num+1):
        list_of_df = [] = [] # concat으로 모으는 df
        for user in tqdm(userIDs):
            unq_testid_by_user = gp_uniq_u_tID[user] # 유저별 testid 배열 
            for j in range(len(unq_testid_by_user)):
                if j-i < 0:
                    temp = grouped.get_group((user,unq_testid_by_user[j])).copy()
                    temp[f'past_testid_{i}'] = -1
                else:
                    temp = grouped.get_group((user,unq_testid_by_user[j])).copy()
                    temp[f'past_testid_{i}'] = unq_testid_by_user[j-i]
                list_of_df.append(temp)
        df_cat = pd.concat(list_of_df)
        df_in[f'past_testid_{i}'] = df_cat[f'past_testid_{i}']
    return df_in

In [18]:
df = past_testid_maker(df,5)

100%|██████████| 7442/7442 [04:03<00:00, 30.62it/s] 


In [17]:
# df[[f'past_testid_{i}' for i in range(1, 6)]] = df_past[[f'past_testid_{i}' for i in range(1, 6)]]

In [23]:
cate = sum([['testId', 'assessmentItemID', 'KnowledgeTag', 'month', 'hour'],
            [f'past_testid_{i}' for i in range(1, 6)],
           ], [])

## 2. Feature Engineering

In [24]:
def feature_engineering(df):    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
    correct_t.columns = ["test_mean", 'test_sum']
    correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
    correct_k.columns = ["tag_mean", 'tag_sum']

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    return df

In [25]:
df = feature_engineering(df)

In [26]:
for c in cate:
    df[c] = df[c].astype('category')

In [28]:
df.to_csv(os.path.join(data_dir,'preprocessed_df.csv'),index=False)